# AutoGluon Assistant - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/autogluon/autogluon-assistant)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://github.com/autogluon/autogluon-assistant)

(Links above are still WIP)

In this tutorial, we will see how to use AutoGluon Assistant (AG-A) to solve machine learning problems **with zero line of code**. AG-A combines the power of AutoGluon's state-of-the-art AutoML capabilities with Large Language Models (LLMs) to automate the entire data science pipeline.

We will cover:
- Setting up AutoGluon Assistant
- Preparing Your Data
- Using AutoGluon Assistant (via Command Line Interface)
- Using AutoGluon Assistant (through Python Programming)
- Using AutoGluon Assistant (via Web UI)

By the end of this tutorial, you'll be able to run your data with our highly accurate ML solutions using just natural language instructions. Let's get started with the installation!

## Setting up AutoGluon Assistant
Getting started with AutoGluon Assistant is straightforward. Let's install it directly using pip:

In [ ]:
!pip install autogluon.assistant

*Warning: If you are using an MacOS, you may need to install libomp with*
```bash
brew install libomp
pip install --upgrade lightgbm
```

AutoGluon Assistant supports two LLM providers: AWS Bedrock (default) and OpenAI. You can configure with our provided tool:
```bash
wget https://raw.githubusercontent.com/autogluon/autogluon-assistant/refs/heads/main/tools/configure_llms.sh
source ./configure_llms.sh
```
Or choose one of the following setups:
```bash
# Option A: AWS Bedrock (Recommended)
export AWS_DEFAULT_REGION='<your-region>'
export AWS_ACCESS_KEY_ID='<your-access-key>'
export AWS_SECRET_ACCESS_KEY='<your-secret-key>'
### OR ###
# Option B: OpenAI
export OPENAI_API_KEY='sk-...'
```

*Note: If using OpenAI, we recommend a paid API key rather than a free-tier account to avoid rate limiting issues.*

Let's verify the installation by importing the package:

In [ ]:
import autogluon.assistant

print(autogluon.assistant.__version__)


Now that you have AutoGluon Assistant installed and configured, let's move on to preparing your data directory structure for your first ML project!

## Preparing Your Data

For this tutorial, we'll use a spaceship transportation dataset which is perfect for getting started with machine learning. The goal is to predict whether an item was transported based on various numerical and categorical features in the dataset. We sampled 1000 training and test examples from the original data. The sampled dataset make this tutorial run quickly, but AutoGluon Assistant can handle the full dataset if desired.

Let's download the example data:

In [ ]:
%%bash
wget https://automl-mm-bench.s3.us-east-1.amazonaws.com/aga/data/aga_sample_data.zip
unzip aga_sample_data.zip

That's it! We now have (under `./toy_data`):

- `train.csv`: Training data with labeled examples
- `test.csv`: Test data for making predictions
- `descriptions.txt`: A description of the dataset and task

Let's take a quick look at our training data and description file:

In [ ]:
import pandas as pd
train_data = pd.read_csv("toy_data/train.csv")
train_data.head()

```
  PassengerId HomePlanet CryoSleep  Destination   Age    VIP  RoomService  FoodCourt  ShoppingMall  Spa  VRDeck                Name  Transported Deck  Cabin_num Side
0     5647_01     Europa      True  TRAPPIST-1e  35.0  False          0.0        NaN           0.0  0.0     NaN  Dyonevi Matoltuble         True    C      178.0    P
1     4061_02       Mars      True  TRAPPIST-1e   0.0  False          0.0        0.0           0.0  0.0     0.0          Graw Kashe         True    F      766.0    S
2     0691_03       Mars      True  TRAPPIST-1e  23.0  False          0.0        0.0           0.0  0.0     0.0          Moss Potte         True    E       49.0    S
3     1094_01      Earth     False  TRAPPIST-1e  60.0    NaN        437.0        2.0           0.0  0.0   365.0   Carona Webstenson        False    F      224.0    P
4     6394_01      Earth      True  TRAPPIST-1e  41.0  False          0.0        0.0           0.0  0.0     0.0   Pattie Lambleyoun         True    G     1037.0    S
```

In [ ]:
with open('toy_data/descriptions.txt', 'r') as f:
    print(f.read())

```
You are solving this data science tasks of binary classification: 
The dataset presented here (the spaceship dataset) comprises a lot of features, including both numerical and categorical features. Some of the features are missing, with nan value. We have splitted the dataset into three parts of train, valid and test. Your task is to predict the Transported item, which is a binary label with True and False. The evaluation metric is the classification accuracy.
```

## Using AutoGluon Assistant (via Command Line Interface)

Now that we have our data ready, let's use AutoGluon Assistant to build our ML model. The simplest way to use AutoGluon Assistant is through the command line - no coding required! After installing the package, you can run it directly from your terminal:

```bash
aga run ./toy_data \
    --presets medium_quality    # (Optional) Choose prediction quality level:
                                # Options: medium_quality, high_quality, best_quality (default)
```

```
INFO:root:Starting AutoGluon-Assistant
INFO:root:Presets: medium_quality
INFO:root:Loading default config from: /media/deephome/autogluon-assistant/src/autogluon/assistant/configs/default.yaml
INFO:root:Merging medium_quality config from: /media/deephome/autogluon-assistant/src/autogluon/assistant/configs/medium_quality.yaml
INFO:root:Successfully loaded config
🤖  Welcome to AutoGluon-Assistant 
Will use task config:
{
    'infer_eval_metric': True,
    'detect_and_drop_id_column': False,
    'task_preprocessors_timeout': 3600,
    'time_limit': 600,
    'save_artifacts': {'enabled': False, 'append_timestamp': True, 'path': './aga-artifacts'},
    'feature_transformers': {
        'enabled_models': [],
        'models': {
            'CAAFE': {
                '_target_': 'autogluon.assistant.transformer.feature_transformers.caafe.CAAFETransformer',
                'eval_model': 'lightgbm',
                'llm_provider': '${llm.provider}',
                'llm_model': '${llm.model}',
                'num_iterations': 5,
                'optimization_metric': 'roc'
            },
            'OpenFE': {'_target_': 'autogluon.assistant.transformer.feature_transformers.openfe.OpenFETransformer', 'n_jobs': 1, 'num_features_to_keep': 10},
            'PretrainedEmbedding': {'_target_': 'autogluon.assistant.transformer.feature_transformers.scentenceFT.PretrainedEmbeddingTransformer', 'model_name': 'all-mpnet-base-v2'}
        }
    },
    'autogluon': {'predictor_init_kwargs': {}, 'predictor_fit_kwargs': {'presets': 'medium_quality'}},
    'llm': {'provider': 'bedrock', 'model': 'anthropic.claude-3-5-sonnet-20241022-v2:0', 'max_tokens': 512, 'proxy_url': None, 'temperature': 0, 'verbose': True}
}
Task path: /media/deephome/autogluon-assistant/toy_data_newest_backup
Task loaded!
TabularPredictionTask(name=toy_data_newest_backup, description=, 3 datasets)
INFO:botocore.credentials:Found credentials from IAM Role: Bedrock_Access
AGA is using model anthropic.claude-3-5-sonnet-20241022-v2:0 from Bedrock to assist you with the task.
INFO:botocore.credentials:Found credentials from IAM Role: Bedrock_Access
INFO:root:It took 0.16 seconds initializing components. Time remaining: 599.83/600.00
Task understanding starts...
description: data_description_file: You are solving this data science tasks of binary classification: \nThe dataset presented here (the spaceship dataset) comprises a lot of features, including both numerical and categorical features. Some of the features are missing, with nan value. We have splitted the dataset into three parts of train, valid and test. Your task is to predict the Transported item, which is a binary label with True and False. The evaluation metric is the classification accuracy.\n
train_data: /media/deephome/autogluon-assistant/toy_data_newest_backup/train.csv
Loaded data from: /media/deephome/autogluon-assistant/toy_data_newest_backup/train.csv | Columns = 16 / 16 | Rows = 1000 -> 1000
test_data: /media/deephome/autogluon-assistant/toy_data_newest_backup/test.csv
Loaded data from: /media/deephome/autogluon-assistant/toy_data_newest_backup/test.csv | Columns = 16 / 16 | Rows = 1000 -> 1000
WARNING: Failed to identify the sample_submission_data of the task, it is set to None.
label_column: Transported
problem_type: binary
eval_metric: accuracy
Total number of prompt tokens: 1614
Total number of completion tokens: 179
Task understanding complete!
Automatic feature generation is disabled. 
INFO:root:It took 17.31 seconds preprocessing task. Time remaining: 582.51/600.00
Model training starts...
Fitting AutoGluon TabularPredictor
predictor_init_kwargs: {'learner_kwargs': {'ignored_columns': []}, 'label': 'Transported', 'problem_type': 'binary', 'eval_metric': 'accuracy'}
predictor_fit_kwargs: {'presets': 'medium_quality'}
No path specified. Models will be saved in: "AutogluonModels/ag-20241119_214901"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #54~20.04.1-Ubuntu SMP Fri Oct 6 22:04:33 UTC 2023
CPU Count:          96
Memory Avail:       1024.83 GB / 1121.80 GB (91.4%)
Disk Space Avail:   63.91 GB / 860.63 GB (7.4%)
===================================================
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 583s
AutoGluon will save models to "AutogluonModels/ag-20241119_214901"
Train Data Rows:    1000
Train Data Columns: 15
Label Column:       Transported
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
        Available Memory:                    1049426.06 MB
        Train Data (Original)  Memory Usage: 0.48 MB (0.0% of available memory)
        Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
        Stage 1 Generators:
                Fitting AsTypeFeatureGenerator...
        Stage 2 Generators:
                Fitting FillNaFeatureGenerator...
        Stage 3 Generators:
                Fitting IdentityFeatureGenerator...
                Fitting CategoryFeatureGenerator...
                        Fitting CategoryMemoryMinimizeFeatureGenerator...
        Stage 4 Generators:
                Fitting DropUniqueFeatureGenerator...
        Stage 5 Generators:
                Fitting DropDuplicatesFeatureGenerator...
        Unused Original Features (Count: 1): ['PassengerId']
                These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
                Features can also be unused if they carry very little information, such as being categorical but having almost entirely unique values or being duplicates of other features.
                These features do not need to be present at inference time.
                ('object', []) : 1 | ['PassengerId']
        Types of features in original data (raw dtype, special dtypes):
                ('float', [])  : 7 | ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', ...]
                ('object', []) : 7 | ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Name', ...]
        Types of features in processed data (raw dtype, special dtypes):
                ('category', []) : 7 | ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Name', ...]
                ('float', [])    : 7 | ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', ...]
        0.1s = Fit runtime
        14 features in original data used to generate 14 features in processed data.
        Train Data (Processed) Memory Usage: 0.06 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.09s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
        To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.2, Train Rows: 800, Val Rows: 200
User-specified model hyperparameters to be fit:
{
        'NN_TORCH': {},
        'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
        'CAT': {},
        'XGB': {},
        'FASTAI': {},
        'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
        'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
        'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif ... Training model for up to 582.42s of the 582.42s of remaining time.
        0.805    = Validation score   (accuracy)
        0.04s    = Training   runtime
        0.04s    = Validation runtime
Fitting model: KNeighborsDist ... Training model for up to 582.34s of the 582.33s of remaining time.
        0.79     = Validation score   (accuracy)
        0.03s    = Training   runtime
        0.04s    = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 582.27s of the 582.27s of remaining time.
        0.83     = Validation score   (accuracy)
        1.44s    = Training   runtime
        0.02s    = Validation runtime

......

Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 556.21s of remaining time.
        Ensemble Weights: {'LightGBMLarge': 0.4, 'NeuralNetTorch': 0.25, 'NeuralNetFastAI': 0.2, 'CatBoost': 0.15}
        0.855    = Validation score   (accuracy)
        0.16s    = Training   runtime
        0.0s     = Validation runtime
AutoGluon training complete, total runtime = 26.47s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 2470.3 rows/s (200 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20241119_214901")
Model training complete!
INFO:root:It took 26.84 seconds training model. Time remaining: 555.67/600.00
Prediction starts...
Prediction complete! Outputs written to aga-output-20241119_214928.csv
INFO:root:It took 0.15 seconds making predictions. Time remaining: 555.52/600.00
```

You can override specific settings in the YAML configuration defined in the [config folder](https://github.com/autogluon/autogluon-assistant/tree/main/src/autogluon/assistant/configs) using
the `config_overrides` parameter with format `"key1=value1, key2.nested=value2"` from the command line.


Here are some example commands on using configuration overrides:

```bash
aga run toy_data --config_overrides "feature_transformers.enabled_models=None, time_limit=3600"

# OR

aga run toy_data --config_overrides "feature_transformers.enabled_models=None" --config_overrides "time_limit=3600"
```

## Using AutoGluon Assistant (through Python Programming)

Let's also look at how to use AutoGluon Assistant programmatically in Python:

In [ ]:
from autogluon.assistant import run_assistant

# Run the assistant
output_file = run_assistant(task_path="./toy_data", presets="medium_quality")

Let's examine the predictions:

In [ ]:
import pandas as pd

predictions = pd.read_csv(output_file)
print("\nFirst few predictions:")
print(predictions.head())

```
First few predictions:
   Transported
0         True
1        False
2         True
3         True
4         True
```

## Using AutoGluon Assistant (via Web UI)

AutoGluon Assistant Web UI allows users to leverage the capabilities of AG-A through an intuitive web interface.

The web UI enables users to upload datasets, configure AG-A runs with customized settings, preview data, monitor execution progress, view and download results, and supports secure, isolated sessions for concurrent users.

### To run the AG-A Web UI:

```bash
aga ui

# OR

# Launch Web-UI on specific port e.g. 8888
aga ui --port 8888
```

AG-A Web UI should now be accessible in your web browser at http://localhost:8501 or the specified port.

*Note: It might take up to a few mins to launch webui for the first time, since we are downloading the sample datasets...*

## Conclusion

In this quickstart tutorial, we saw how AutoGluon Assistant simplifies the entire ML pipeline by allowing users to solve machine learning problems with minimal efforts. With just a data directory, AutoGluon Assistant handles the entire process from data understanding to prediction generation. Check out the other tutorials (WIP) to learn more about customizing the configuration, using different LLM providers, and handling various types of ML tasks.

Want to dive deeper? Explore our GitHub repository for more advanced features and examples.